In [121]:
!pip install torch torchvision transformers pillow groq -q

In [ ]:
import os
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification
from google.colab import files
from groq import Groq


from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found. Please set it in Colab secrets.")

client = Groq(api_key=GROQ_API_KEY)

model_name = "Ateeqq/ai-vs-human-image-detector"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)
model.eval()


print("Upload your image file (.jpg, .png, etc.)")
uploaded = files.upload()
image_path = next(iter(uploaded.keys()))


image = Image.open(image_path).convert("RGB")
inputs = processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits


temperature = 6.0  
probabilities = torch.nn.functional.softmax(logits / temperature, dim=-1)
ai_probability = probabilities[0][0].item()
human_probability = probabilities[0][1].item()


diff = abs(ai_probability - human_probability)
if diff >= 0.7:
    confidence = "High"
elif diff >= 0.3:
    confidence = "Medium"
else:
    confidence = "Low"

result_label = "AI-generated" if ai_probability > human_probability else "Human-generated"


prompt = f"""
You are an AI image analysis expert.
The detector outputs:
- AI-generated probability: {ai_probability:.4f}
- Human-generated probability: {human_probability:.4f}
- Confidence level: {confidence}

Give a short, human-readable explanation (1–2 sentences) of why the image was likely classified as {result_label}.
Base it on visual cues such as texture, lighting, symmetry, artifacts, or other hints you might infer.
Avoid repeating probabilities; focus on reasoning only.
"""

response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.6,
)

explanation = response.choices[0].message.content.strip()

print("\n🖼 Image Detection Summary:")
print(f"🤖 AI-generated Probability: {ai_probability:.4f}")

print(f"🎯 Confidence Level: {confidence}")
print(f"🗣 Explanation: {explanation}")

Upload your image file (.jpg, .png, etc.)


Saving f1.jpeg to f1 (2).jpeg

 AI-generated Probability: 0.2653
 Confidence Level: Medium
 Explanation: The image was likely classified as human-generated due to its natural texture and lighting, which may have appeared more organic and nuanced than what is typically produced by AI algorithms. The presence of subtle asymmetries and imperfections in the image may have also contributed to this classification, as these characteristics are often harder for AI generators to replicate convincingly.
